In [13]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device
#import wandb

In [14]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
#cfg = Config.fromfile('./configs/_custom_/convnext/cascade_mask_rcnn_convnext_base_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco_in22k.py')
cfg = Config.fromfile('./configs/_custom_/convnext/cascade_mask_rcnn_convnext_xlarge_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco_in22k_detr.py')

root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'stratified_v2/train_fold_1.json' # train json 정보
#cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'stratified_v2/val_fold_1.json'
#cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보

cfg.train_pipeline = cfg.train_pipeline
cfg.val_pipeline = cfg.test_pipeline
cfg.test_pipeline = cfg.test_pipeline

cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.val.pipeline = cfg.val_pipeline
cfg.data.test.pipeline = cfg.test_pipeline

cfg.data.samples_per_gpu = 1
cfg.data.workers_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/retinanet_swin-t_trash'

#cfg.model.roi_head.bbox_head.num_classes = 10
cfg.optimizer_config.grad_clip = None #dict(max_norm=35, norm_type=2)
cfg.optimizer.lr=0.000005
#cfg.lr_config.step=[4]
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
log_config = dict(
    interval=50,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(
            type='WandbLoggerHook',
            init_kwargs=dict(project='Model', name=f'{name}', entity='hi-ai'),
            #interval=10,
            log_checkpoint=True,
            log_checkpoint_metadata=True,
            num_eval_images=0
            )])
# for wandb
cfg.log_config = log_config


#cfg.evaluation = dict(interval=1, start=1,save_best='auto')
#cfg.evaluation = dict(interval=1, save_best='bbox_mAP_50', metric='bbox')
#cfg.evaluation = dict(interval=1, save_best='bbox_mAP_50', metric='mAP')
# cfg.evaluation['save_best'] = 'bbox_mAP_50' 
#cfg.evaluation.metric = 'mAP'

cfg.device = get_device()
#cfg.runner = dict(type='EpochBasedRunner', max_epochs=12)

In [15]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [16]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 3914, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3161  | 1 [Paper]     | 5115  | 2 [Paper pack]  | 706   | 3 [Metal]   | 769   | 4 [Glass]    | 835   |
| 5 [Plastic]       | 2350  | 6 [Styrofoam] | 1026  | 7 [Plastic bag] | 4151  | 8 [Battery] | 143   | 9 [Clothing] | 377   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [17]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2023-05-11 10:08:45,766 - mmdet - INFO - load checkpoint from http path: https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth
2023-05-11 10:08:45,913 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2023-05-11 10:08:45,951 - mmcv - INFO - initialize RetinaHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01, 'override': {'type': 'Normal', 'name': 'retina_cls', 'std': 0.01, 'bias_prob': 0.01}}
2023-05-11 10:08:46,022 - mmcv - INFO - 
backbone.patch_embed.projection.weight - torch.Size([96, 3, 4, 4]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
2023-05-11 10:08:46,023 - mmcv - INFO - 
backbone.patch_embed.projection.bias - torch.Size([96]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
2023-05-11 10:08:46,024 - mmcv - INFO - 
backbone.patch_embed.norm.weight - torch.Size([96]): 
Initialized by user-defined `init_weights` 

In [18]:
# 모델 학습
#train_detector(model, datasets[0], cfg, distributed=False, validate=False)
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2023-05-11 10:08:46,340 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2023-05-11 10:08:46,426 - mmdet - INFO - Start running, host: root@bbe3323c95ba, work_dir: /opt/ml/baseline/mmdetection/work_dirs/retinanet_swin-t_trash
2023-05-11 10:08:46,427 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2023-05-11 10:09:05,408 - mmdet - INFO - Epoch [1][50/3914]	lr: 7.205e-07, eta: 7:55:16, time: 0.202, data_time: 0.052, memory: 9219, loss_cls: 1.1398, loss_bbox: 0.6841, loss: 1.8239
2023-05-11 10:09:13,401 - mmdet - INFO - Epoch [1][100/3914]	lr: 9.455e-07, eta: 7:05:07, time: 0.160, data_time: 0.004, memory: 9219, loss_cls: 1.1410, loss_bbox: 0.6687, loss: 1.8097
2023-05-11 10:09:21,186 - mmdet - INFO - Epoch [1][150/3914]	lr: 1.170e-06, eta: 6:45:02, time: 0.156, data_time: 0.004, memory: 9219, loss_cls: 1.1368, loss_bbox: 0.6670, loss: 1.8038
2023-05-11 10:09:29,167 - mmdet - INFO - Epoch [1][200/3914]	lr: 1.395e-06, eta: 6:37:14, time: 0.160, data_time: 0.004, memory: 9219, loss_cls: 1.1377, loss_bbox: 0.6617, loss: 1.7994
2023-05-11 10:09:36,905 - mmdet - INFO - Epoch [1][250/3914]	lr: 1.620e-06, eta: 6:30:13, time: 0.155, data_time: 0.004, memory: 9219, loss_cls: 1.1357, loss_bbox: 0.6731, loss: 1.8088
2023-05-11 10:09:44,897 - mmdet - INFO - Epoch [1][300/3914]	lr: 1.845e-06, 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 969/969, 18.3 task/s, elapsed: 53s, ETA:     0s

2023-05-11 10:20:09,324 - mmdet - INFO - Evaluating bbox...


...

2023-05-11 17:07:36,420 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.39s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.13s).
Accumulating evaluation results...


2023-05-11 17:07:45,314 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.255
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.414
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.264
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.014
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.036
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.303
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.476
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.476
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.476
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.034
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.156
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=1.85s).


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
learning_rate,███████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▆▅▅▄▄▄▄▃▃▂▂▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▂
train/loss_bbox,█▅▅▄▃▃▃▄▃▂▂▂▂▃▂▂▂▂▂▁▂▂▁▁▁▁▁▂▁▂▁▂▂▂▂▂▂▁▁▂
train/loss_cls,█▆▆▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▂▁▁▁▂▁▁▁▁▂▂
val/bbox_mAP,▁▁▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇█▇██████████
val/bbox_mAP_50,▁▁▂▂▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇█▇████████████
val/bbox_mAP_75,▁▁▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇████████
val/bbox_mAP_l,▁▁▂▂▃▃▄▄▄▅▄▅▆▆▆▆▆▇▇▇▇▇▇▇████████████
val/bbox_mAP_m,▁▂▂▃▃▄▄▄▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇██████████
